In [6]:
import pandas as pd
import sys

%load_ext autoreload
%autoreload 2
sys.path.insert(1, '../src/')

from guara_unsupervised_modelz import *
import utils

pd.set_option('display.max_columns', None)

In [7]:
df = utils.import_s3_example_dataset('bombas')[['date',
                                                'vibracao_ponto_1',
                                                'vibracao_ponto_2',
                                                'vibracao_ponto_3',
                                                'vibracao_ponto_4',
                                                'vibracao_ponto_5'
                                               ]]
df.head()

Baixando dataset BOMBAS do S3...


,date,vibracao_ponto_1,vibracao_ponto_2,vibracao_ponto_3,vibracao_ponto_4,vibracao_ponto_5
0,2017-12-28 15:00:00,1.150845,0.983295,2.066202,4.444914,4.024140
1,2017-12-28 15:05:00,1.152106,0.980713,2.067280,4.458808,4.021726
2,2017-12-28 15:10:00,1.216263,0.962285,2.037572,4.539507,3.985949
3,2017-12-28 15:15:00,1.482624,0.761433,1.733252,5.693767,3.722689
4,2017-12-28 15:20:00,1.482012,0.763680,1.734059,5.697873,3.715978


In [8]:
train_split = 0.8
n_train_split = int(train_split*len(df))

Xtrain = df.iloc[:n_train_split, :].copy()
Xval = df.iloc[n_train_split:, :].copy()

Xt_date = Xtrain.date.copy()
Xv_date = Xval.date.copy()

Xtrain = Xtrain.drop('date', axis = 1)
Xval = Xval.drop('date', axis = 1)

print(Xtrain.shape)
print(Xval.shape)
Xtrain.head()

(163900, 5)
(40976, 5)


,vibracao_ponto_1,vibracao_ponto_2,vibracao_ponto_3,vibracao_ponto_4,vibracao_ponto_5
0,1.150845,0.983295,2.066202,4.444914,4.024140
1,1.152106,0.980713,2.067280,4.458808,4.021726
2,1.216263,0.962285,2.037572,4.539507,3.985949
3,1.482624,0.761433,1.733252,5.693767,3.722689
4,1.482012,0.763680,1.734059,5.697873,3.715978


In [10]:
md = UnsupervisedModelz('gmm')
Xtrain = md.preprocess_fit_transform(Xtrain)
Xtrain.head()

,vibracao_ponto_1,vibracao_ponto_2,vibracao_ponto_3,vibracao_ponto_4,vibracao_ponto_5
0,0.095807,0.147999,0.057597,0.243702,0.195596
1,0.095912,0.147611,0.057627,0.244464,0.195479
2,0.101253,0.144837,0.056798,0.248889,0.193740
3,0.123427,0.114606,0.048315,0.312174,0.180944
4,0.123376,0.114944,0.048338,0.312399,0.180618


In [13]:
params = {
    'max_iter': 100, 
    'n_init': 3,
}

sr = md.search(Xtrain.sample(10000), params, min_clusters = 3, max_clusters = 7)

Realizando busca com o modelo GMM...
Fitted n_clusters = 3 (6.16 s)
Fitted n_clusters = 4 (6.17 s)
Fitted n_clusters = 5 (6.93 s)
Fitted n_clusters = 6 (7.12 s)
Fitted n_clusters = 7 (7.44 s)

Busca feita em 0.56 minutos


In [14]:
list(range(min_clusters,max_clusters+1))

[2, 3, 4, 5]

In [45]:
import concurrent.futures
import time
from itertools import repeat

start = time.perf_counter()


def do_something(payload):
    a = payload[0]
    b = payload[1]
    print(f'Sleeping {a} second(s)...')
    time.sleep(a)
    return b

a = 3
with concurrent.futures.ProcessPoolExecutor() as executor:
    secs = [(2,1),(2,1),(2,1),(2,1)]
    results = executor.map(do_something,secs)
    
    
finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Sleeping 2 second(s)...Sleeping 2 second(s)...Sleeping 2 second(s)...Sleeping 2 second(s)...



Finished in 2.14 second(s)


In [46]:
[i for i in results]

[1, 1, 1, 1]

In [33]:
a, b = (1,2)

In [47]:
def fit_gmm(payload):
    '''
    Fit GMM model 
    (https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html)
    '''
    X = payload[1] 
    n_clusters = payload[0] 
    
    params = {
        'max_iter': 100, 
        'n_init': 3,
        'n_components':n_clusters
    }
    
    gmm = GMM(**params).fit(X) 


    pred = gmm.predict(X)
    pred = pd.Series(pred, index = X.index)

    # get centroids
    centroids = pd.DataFrame(gmm.means_, columns = X.columns)

    # get metrics  
    metrics = {}
    metrics['BIC'] = gmm.bic(X)
    metrics['AIC'] = gmm.aic(X)
    try:metrics['silhouette_score'] = silhouette_score(X, pred, metric = 'mahalanobis')
    except: metrics['silhouette_score'] = None
    try: metrics['davies_bouldin_score'] = davies_bouldin_score(X, pred)
    except: metrics['davies_bouldin_score'] = None

    train_info = {}
    train_info['hyperparameters'] = gmm.get_params()
    train_info['n_clusters'] = gmm.get_params()['n_components']
    train_info['metrics'] = metrics
    train_info['centroids'] = centroids

    return train_info#, gmm

In [56]:
t1 = time.time()
with concurrent.futures.ProcessPoolExecutor() as executor:
    it = [(i, Xtrain.sample(1000)) for i in range(2,10)]
    results = executor.map(fit_gmm,it)
    
t2 = time.time()

KeyboardInterrupt: 

In [ ]:
[i for i in results]

In [55]:
t1_ = time.time()

it = [(i, Xtrain.sample(1000)) for i in range(2,10)]
for payload in it:
    k= fit_gmm(payload)
    print(k)
t2_ = time.time()

{'hyperparameters': {'covariance_type': 'full', 'init_params': 'kmeans', 'max_iter': 100, 'means_init': None, 'n_components': 2, 'n_init': 3, 'precisions_init': None, 'random_state': None, 'reg_covar': 1e-06, 'tol': 0.001, 'verbose': 0, 'verbose_interval': 10, 'warm_start': False, 'weights_init': None}, 'n_clusters': 2, 'metrics': {'BIC': -13623.440932059011, 'AIC': -13824.658898497279, 'silhouette_score': 0.30812045260880494, 'davies_bouldin_score': 0.6236009772547249}, 'centroids':    vibracao_ponto_1  vibracao_ponto_2  vibracao_ponto_3  vibracao_ponto_4  \
0          0.077622          0.103787          0.028636          0.171351   
1          0.116976          0.161714          0.186046          0.613719   

   vibracao_ponto_5  
0          0.172963  
1          0.408290  }
{'hyperparameters': {'covariance_type': 'full', 'init_params': 'kmeans', 'max_iter': 100, 'means_init': None, 'n_components': 3, 'n_init': 3, 'precisions_init': None, 'random_state': None, 'reg_covar': 1e-06, 'to

In [ ]:
(t2-t1)/(t2_-t1_)

In [26]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import pandas as pd
from pandas.api.types import is_string_dtype
import numpy as np
import datetime
import pickle
import warnings
import os
from time import time
from tabulate import tabulate
import itertools
import plotly.express as px
import shap
from kmodes.kprototypes import KPrototypes
from tabulate import tabulate

#Sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score 
from sklearn.metrics import davies_bouldin_score
from sklearn.mixture import GaussianMixture as GMM
from sklearn.ensemble import IsolationForest
